<a href="https://colab.research.google.com/github/hammadkhann/Effective-Dense-Retrieval/blob/main/Neural_Reranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pyterrier Setup**

In [2]:
!pip -q install python-terrier
import pyterrier as pt
pt.init()

PyTerrier 0.7.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)


## **Importing useful libraries**



In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pickle
from google.colab import drive
from pyterrier.measures import *

pd.options.display.max_rows = None
pd.options.display.max_columns = None

## **Connect Google Drive**


In [6]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Preparing testing dataset**

## **DPH(Sparse Retrieval)**

## TREC-2019

In [9]:
br = pt.BatchRetrieve.from_dataset('msmarco_passage', 'terrier_stemmed_text', num_results=100, metadata = ['docno', 'text'], verbose=True)

22:10:09.058 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.


In [10]:
dph_2019 = pd.DataFrame()
topics = pt.get_dataset('msmarco_passage').get_topics('test-2019')
topics = pt.get_dataset('msmarco_passage').get_qrels('test-2019')[["qid"]].drop_duplicates().merge(topics, on=['qid'])
for res in br.transform_gen(topics, batch_size=10):
  dph_2019 = dph_2019.append(res)
print("Number of topics: ", len(topics))

BR(DPH):   0%|          | 0/10 [00:00<?, ?q/s]

BR(DPH):   0%|          | 0/10 [00:00<?, ?q/s]

BR(DPH):   0%|          | 0/10 [00:00<?, ?q/s]

BR(DPH):   0%|          | 0/10 [00:00<?, ?q/s]

BR(DPH):   0%|          | 0/3 [00:00<?, ?q/s]

Number of topics:  43


In [11]:
dph_2019.shape

(4205, 7)

## TREC-2020

In [12]:
dph_2020 = pd.DataFrame()
topics = pt.get_dataset('msmarco_passage').get_topics('test-2020')
topics = pt.get_dataset('msmarco_passage').get_qrels('test-2020')[["qid"]].drop_duplicates().merge(topics, on=['qid'])
for res in br.transform_gen(topics, batch_size=10):
  dph_2020 = dph_2020.append(res)
  
print("Number of topics: ", len(topics))

BR(DPH):   0%|          | 0/10 [00:00<?, ?q/s]

BR(DPH):   0%|          | 0/10 [00:00<?, ?q/s]

BR(DPH):   0%|          | 0/10 [00:00<?, ?q/s]

BR(DPH):   0%|          | 0/10 [00:00<?, ?q/s]

BR(DPH):   0%|          | 0/10 [00:00<?, ?q/s]

BR(DPH):   0%|          | 0/4 [00:00<?, ?q/s]

Number of topics:  54


In [13]:
dph_2020.shape

(5329, 7)

## **ColBERT(Dense Retrieval)**

## TREC-2019

In [14]:
index = pt.IndexFactory.of(pt.get_dataset("msmarco_passage").get_index("terrier_stemmed_text"))
print(index.getCollectionStatistics())

22:10:27.358 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.
Number of documents: 8841823
Number of terms: 1170682
Number of postings: 215238456
Number of fields: 1
Number of tokens: 288759529
Field names: [text]
Positions:   false



In [15]:
colbert_2019 = pt.io.read_results("/content/drive/MyDrive/E2E.2019.res")
# get document text for each docid
meta = index.getMetaIndex()
text = []
for index, row in colbert_2019.iterrows():
    text.append(meta.getItem('text', int(row['docno'])))
colbert_2019['text'] = text

# get query for each qid
topics_2019 = pt.get_dataset('msmarco_passage').get_topics('test-2019')
colbert_2019 = pd.merge(colbert_2019, topics_2019, on='qid', how='left')

# filter queries on 43 topics
colbert_2019 = colbert_2019[colbert_2019['qid'].isin(dph_2019['qid'].unique())]
# filter data on top100 results for each 43 topics
colbert_2019 = colbert_2019.groupby('qid').apply(lambda x : x.sort_values(by = 'score', ascending = False).head(100).reset_index(drop = True))
# reset index
colbert_2019.reset_index(drop=True, inplace=True)
colbert_2019.shape

(4300, 7)

## TREC-2020

In [16]:
colbert_2020 = pt.io.read_results("/content/drive/MyDrive/E2E.2020.res")
index = pt.IndexFactory.of(pt.get_dataset("msmarco_passage").get_index("terrier_stemmed_text"))

# get document text for each docid
meta = index.getMetaIndex()
text = []
for index, row in colbert_2020.iterrows():
    text.append(meta.getItem('text', int(row['docno'])))
colbert_2020['text'] = text

# get query for each qid
topics_2020 = pt.get_dataset('msmarco_passage').get_topics('test-2020')
colbert_2020 = pd.merge(colbert_2020, topics_2020, on='qid', how='left')

# filter queries on 43 topics
colbert_2020 = colbert_2020[colbert_2020['qid'].isin(dph_2020['qid'].unique())]
# filter data on top100 results for each 43 topics
colbert_2020 = colbert_2020.groupby('qid').apply(lambda x : x.sort_values(by = 'score', ascending = False).head(100).reset_index(drop = True))
# reset index
colbert_2020.reset_index(drop=True, inplace=True)
colbert_2020.shape

22:13:19.654 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.


(5400, 7)

# **Pyterrier-ColBERT Setup**

In [17]:
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_colbert.git

  Cloning https://github.com/terrierteam/pyterrier_colbert.git to /tmp/pip-req-build-ca_8jtdc
  Running command git clone -q https://github.com/terrierteam/pyterrier_colbert.git /tmp/pip-req-build-ca_8jtdc
  Cloning https://github.com/cmacdonald/ColBERT.git (to revision v0.2) to /tmp/pip-install-755nypjp/colbert_6725adb8421e45ee86a2dfa8d71c6afb
  Running command git clone -q https://github.com/cmacdonald/ColBERT.git /tmp/pip-install-755nypjp/colbert_6725adb8421e45ee86a2dfa8d71c6afb
  Running command git checkout -b v0.2 --track origin/v0.2
  Switched to a new branch 'v0.2'
  Branch 'v0.2' set up to track remote branch 'v0.2' from 'origin'.


In [18]:
from pyterrier_colbert.ranking import ColBERTFactory

## Updating pyterrier-ColBERT load methods to support latest version of transformers library.

In [19]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs
!git lfs install
!git clone https://huggingface.co/Luyu/condenser

Detected operating system as Ubuntu/bionic.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (3.0.2).
0 upgraded, 0 newly installed, 0 to remove and 43 not upgraded.
Git LFS initialized.
fatal: destination path 'condenser' already exists and is not an empty directory.


In [20]:
import os
import ujson
import torch
import random

from collections import defaultdict, OrderedDict

from colbert.parameters import DEVICE
import colbert.utils.utils
from colbert.utils.utils import print_message


def _load_checkpoint(path, model, optimizer=None, do_print=True):
    if do_print:
        print_message("#> Loading checkpoint", path)

    checkpoint = torch.load(path, map_location='cpu')

    state_dict = checkpoint['model_state_dict']
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k
        if k[:7] == 'module.':
            name = k[7:]
        new_state_dict[name] = v

    checkpoint['model_state_dict'] = new_state_dict

    model.load_state_dict(checkpoint['model_state_dict'], strict=False)

    if optimizer:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'], strict=False)

    if do_print:
        print_message("#> checkpoint['epoch'] =", checkpoint['epoch'])
        print_message("#> checkpoint['batch'] =", checkpoint['batch'])

    return checkpoint

colbert.utils.utils.load_checkpoint = _load_checkpoint
colbert.utils.utils.load_checkpoint.__globals__['load_checkpoint'] = _load_checkpoint
from colbert.modeling.colbert import ColBERT

def _load_model(args, do_print=True):
    colbert = ColBERT.from_pretrained('/content/condenser/',
                                      query_maxlen=args.query_maxlen,
                                      doc_maxlen=args.doc_maxlen,
                                      dim=args.dim,
                                      similarity_metric=args.similarity,
                                      mask_punctuation=args.mask_punctuation)
    colbert = colbert.to(DEVICE)

    print_message("#> Loading model checkpoint.", condition=do_print)

    checkpoint = load_checkpoint(args.checkpoint, colbert, do_print=do_print)

    colbert.eval()

    return colbert, checkpoint

In [21]:
import colbert.evaluation.load_model
import pyterrier_colbert.ranking
colbert.evaluation.load_model.load_model = _load_model
pyterrier_colbert.ranking.load_model = _load_model

# **Evaluation on TREC 2019**

## **Loading model checkpoints**

### Baseline

In [23]:
colbert_checkpoint="/content/drive/MyDrive/colbert-baseline-50000.dnn" 
colbert_factory = ColBERTFactory(colbert_checkpoint, None, None)
colbert_pipeline_ann = pt.transformer.SourceTransformer(colbert_2019) >>  colbert_factory.text_scorer()
colbert_pipeline = pt.transformer.SourceTransformer(dph_2019) >>  colbert_factory.text_scorer()

/usr/local/lib/python3.7/dist-packages/pyterrier_colbert/ranking.py:233: UserWarning: No index_root and index_name specified - no index ranking possible
  warn("No index_root and index_name specified - no index ranking possible")
/usr/local/lib/python3.7/dist-packages/pyterrier_colbert/ranking.py:249: UserWarning: Faiss not installed. You cannot do retrieval
  warn("Faiss not installed. You cannot do retrieval")
Some weights of the model checkpoint at /content/condenser/ were not used when initializing ColBERT: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'bert.embeddings.position_ids']
- This IS expected if you are initializing ColBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification mo

[Dec 01, 22:20:27] #> Loading model checkpoint.
[Dec 01, 22:20:27] #> Loading checkpoint /content/drive/MyDrive/colbert-baseline-50000.dnn
[Dec 01, 22:20:48] #> checkpoint['epoch'] = 0
[Dec 01, 22:20:48] #> checkpoint['batch'] = 50000


### RD1: Additional signal during ColBERT training

In [26]:
cls_similarity_checkpoint="/content/drive/MyDrive/colbert-increase-similarity.dnn" 
cls_similarity_factory = ColBERTFactory(cls_similarity_checkpoint, None, None)
cls_similarity_pipeline_ann = pt.transformer.SourceTransformer(colbert_2019) >>  cls_similarity_factory.text_scorer()
cls_similarity_pipeline = pt.transformer.SourceTransformer(dph_2019) >>  cls_similarity_factory.text_scorer()

/usr/local/lib/python3.7/dist-packages/pyterrier_colbert/ranking.py:233: UserWarning: No index_root and index_name specified - no index ranking possible
  warn("No index_root and index_name specified - no index ranking possible")
/usr/local/lib/python3.7/dist-packages/pyterrier_colbert/ranking.py:249: UserWarning: Faiss not installed. You cannot do retrieval
  warn("Faiss not installed. You cannot do retrieval")
Some weights of the model checkpoint at /content/condenser/ were not used when initializing ColBERT: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'bert.embeddings.position_ids']
- This IS expected if you are initializing ColBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification mo

[Dec 01, 22:25:07] #> Loading model checkpoint.
[Dec 01, 22:25:07] #> Loading checkpoint /content/drive/MyDrive/colbert-increase-similarity.dnn
[Dec 01, 22:25:29] #> checkpoint['epoch'] = 0
[Dec 01, 22:25:29] #> checkpoint['batch'] = 50000


In [27]:
token_overlap_checkpoint="/content/drive/MyDrive/colbert-token-overlap-50000.dnn" 
token_overlap_factory = ColBERTFactory(token_overlap_checkpoint, None, None)
token_overlap_pipeline_ann = pt.transformer.SourceTransformer(colbert_2019) >>  token_overlap_factory.text_scorer()
token_overlap_pipeline = pt.transformer.SourceTransformer(dph_2019) >>  token_overlap_factory.text_scorer()

/usr/local/lib/python3.7/dist-packages/pyterrier_colbert/ranking.py:233: UserWarning: No index_root and index_name specified - no index ranking possible
  warn("No index_root and index_name specified - no index ranking possible")
/usr/local/lib/python3.7/dist-packages/pyterrier_colbert/ranking.py:249: UserWarning: Faiss not installed. You cannot do retrieval
  warn("Faiss not installed. You cannot do retrieval")
Some weights of the model checkpoint at /content/condenser/ were not used when initializing ColBERT: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'bert.embeddings.position_ids']
- This IS expected if you are initializing ColBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification mo

[Dec 01, 22:25:33] #> Loading model checkpoint.
[Dec 01, 22:25:33] #> Loading checkpoint /content/drive/MyDrive/colbert-token-overlap-50000.dnn
[Dec 01, 22:25:54] #> checkpoint['epoch'] = 0
[Dec 01, 22:25:54] #> checkpoint['batch'] = 50000


### RD2: Changing Underlying Pre-trained Model 

In [28]:
coltacl_checkpoint="/content/drive/MyDrive/colTaCL-50000.dnn" 
coltacl_factory = ColBERTFactory(coltacl_checkpoint, None, None)
coltacl_pipeline_ann = pt.transformer.SourceTransformer(colbert_2019) >>  coltacl_factory.text_scorer()
coltacl_pipeline = pt.transformer.SourceTransformer(dph_2019) >>  coltacl_factory.text_scorer()

/usr/local/lib/python3.7/dist-packages/pyterrier_colbert/ranking.py:233: UserWarning: No index_root and index_name specified - no index ranking possible
  warn("No index_root and index_name specified - no index ranking possible")
/usr/local/lib/python3.7/dist-packages/pyterrier_colbert/ranking.py:249: UserWarning: Faiss not installed. You cannot do retrieval
  warn("Faiss not installed. You cannot do retrieval")
Some weights of the model checkpoint at /content/condenser/ were not used when initializing ColBERT: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'bert.embeddings.position_ids']
- This IS expected if you are initializing ColBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification mo

[Dec 01, 22:25:58] #> Loading model checkpoint.
[Dec 01, 22:25:58] #> Loading checkpoint /content/drive/MyDrive/colTaCL-50000.dnn
[Dec 01, 22:26:31] #> checkpoint['epoch'] = 0
[Dec 01, 22:26:31] #> checkpoint['batch'] = 50000


In [29]:
colCondenser_checkpoint="/content/drive/MyDrive/colCondenser-50000.dnn" 
colCondenser_factory = ColBERTFactory(colCondenser_checkpoint, None, None)
colCondenser_pipeline_ann = pt.transformer.SourceTransformer(colbert_2019) >>  colCondenser_factory.text_scorer()
colCondenser_pipeline = pt.transformer.SourceTransformer(dph_2019) >>  colCondenser_factory.text_scorer()

/usr/local/lib/python3.7/dist-packages/pyterrier_colbert/ranking.py:233: UserWarning: No index_root and index_name specified - no index ranking possible
  warn("No index_root and index_name specified - no index ranking possible")
/usr/local/lib/python3.7/dist-packages/pyterrier_colbert/ranking.py:249: UserWarning: Faiss not installed. You cannot do retrieval
  warn("Faiss not installed. You cannot do retrieval")
Some weights of the model checkpoint at /content/condenser/ were not used when initializing ColBERT: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'bert.embeddings.position_ids']
- This IS expected if you are initializing ColBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification mo

[Dec 01, 22:26:35] #> Loading model checkpoint.
[Dec 01, 22:26:35] #> Loading checkpoint /content/drive/MyDrive/colCondenser-50000.dnn
[Dec 01, 22:27:10] #> checkpoint['epoch'] = 0
[Dec 01, 22:27:10] #> checkpoint['batch'] = 50000


### RD3: Multi-Task Learning (MTL)   

In [30]:
mtl_checkpoint="/content/drive/MyDrive/mtl_v2_cls+tok_losses-50000.dnn" 
mtl_factory = ColBERTFactory(mtl_checkpoint, None, None)
mtl_pipeline_ann = pt.transformer.SourceTransformer(colbert_2019) >>  mtl_factory.text_scorer()
mtl_pipeline = pt.transformer.SourceTransformer(dph_2019) >>  mtl_factory.text_scorer()

/usr/local/lib/python3.7/dist-packages/pyterrier_colbert/ranking.py:233: UserWarning: No index_root and index_name specified - no index ranking possible
  warn("No index_root and index_name specified - no index ranking possible")
/usr/local/lib/python3.7/dist-packages/pyterrier_colbert/ranking.py:249: UserWarning: Faiss not installed. You cannot do retrieval
  warn("Faiss not installed. You cannot do retrieval")
Some weights of the model checkpoint at /content/condenser/ were not used when initializing ColBERT: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'bert.embeddings.position_ids']
- This IS expected if you are initializing ColBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification mo

[Dec 01, 22:27:14] #> Loading model checkpoint.
[Dec 01, 22:27:14] #> Loading checkpoint /content/drive/MyDrive/mtl_v2_cls+tok_losses-50000.dnn
[Dec 01, 22:27:47] #> checkpoint['epoch'] = 0
[Dec 01, 22:27:47] #> checkpoint['batch'] = 50000


## **Experiment**

### DPH(Sparse Retrieval) Re-ranking

In [25]:
pt.Experiment(
    [colbert_pipeline, cls_similarity_pipeline, token_overlap_pipeline, colCondenser_pipeline, coltacl_pipeline, mtl_pipeline],
    pt.get_dataset('msmarco_passage').get_topics('test-2019'),
    pt.get_dataset('msmarco_passage').get_qrels('test-2019'),
    filter_by_qrels=True,
    names=['DPH >> ColBERT', 'DPH >> ColBERT-increase-cls-similarity', 'DPH >> ColBERT-add-token-overlap', 'DPH >> ColCondenser', 'DPH >> ColTaCL', 'DPH >> ColBERT-mtl-objective'],
    eval_metrics= [nDCG@10, AP(rel=2)@100, RR(rel=2)@10]
)

,name,nDCG@10,AP(rel=2)@100,RR(rel=2)@10
0,DPH >> ColBERT,0.655238,0.337195,0.834884
1,DPH >> ColBERT-increase-cls-similarity,0.669187,0.340263,0.865725
2,DPH >> ColBERT-add-token-overlap,0.666822,0.340682,0.810299
3,DPH >> ColCondenser,0.662346,0.345593,0.824612
4,DPH >> ColTaCL,0.660369,0.334005,0.851550
5,DPH >> ColBERT-mtl-objective,0.640291,0.335387,0.868605


### ColBERT(Dense Retrieval) Re-ranking

In [26]:
pt.Experiment(
    [colbert_pipeline_ann, cls_similarity_pipeline_ann, token_overlap_pipeline_ann, colCondenser_pipeline_ann, coltacl_pipeline_ann, mtl_pipeline_ann],
    pt.get_dataset('msmarco_passage').get_topics('test-2019'),
    pt.get_dataset('msmarco_passage').get_qrels('test-2019'),
    filter_by_qrels=True,
    names=['ANN-ColBERT >> ColBERT', 'ANN-ColBERT >> ColBERT-increase-cls-similarity', 'ANN-ColBERT >> ColBERT-add-token-overlap', 'ANN-ColBERT >> ColCondenser', 'ANN-ColBERT >> ColTaCL', 'ANN-ColBERT >> ColBERT-mtl-objective'],
    eval_metrics= [nDCG@10, AP(rel=2)@100, RR(rel=2)@10]
)

,name,nDCG@10,AP(rel=2)@100,RR(rel=2)@10
0,ANN-ColBERT >> ColBERT,0.691909,0.391189,0.858527
1,ANN-ColBERT >> ColBERT-increase-cls-similarity,0.705811,0.400619,0.889535
2,ANN-ColBERT >> ColBERT-add-token-overlap,0.671373,0.383359,0.825581
3,ANN-ColBERT >> ColCondenser,0.680013,0.397421,0.837209
4,ANN-ColBERT >> ColTaCL,0.683721,0.379544,0.854651
5,ANN-ColBERT >> ColBERT-mtl-objective,0.640436,0.354623,0.862403


# **Evaluation on TREC 2020**

## **Loading model checkpoints**

### Baseline 

In [24]:
colbert_pipeline_ann_2020 = pt.transformer.SourceTransformer(colbert_2020) >>  colbert_factory.text_scorer()
colbert_pipeline_2020 = pt.transformer.SourceTransformer(dph_2020) >>  colbert_factory.text_scorer()

### RD1: Additional signal during ColBERT training

In [32]:
cls_similarity_pipeline_ann_2020 = pt.transformer.SourceTransformer(colbert_2020) >>  cls_similarity_factory.text_scorer()
cls_similarity_pipeline_2020 = pt.transformer.SourceTransformer(dph_2020) >>  cls_similarity_factory.text_scorer()

In [33]:
token_overlap_pipeline_ann_2020 = pt.transformer.SourceTransformer(colbert_2020) >>  token_overlap_factory.text_scorer()
token_overlap_pipeline_2020 = pt.transformer.SourceTransformer(dph_2020) >>  token_overlap_factory.text_scorer()

### RD2: Changing Underlying Pre-trained Model 

In [34]:
coltacl_pipeline_ann_2020 = pt.transformer.SourceTransformer(colbert_2020) >>  coltacl_factory.text_scorer()
coltacl_pipeline_2020 = pt.transformer.SourceTransformer(dph_2020) >>  coltacl_factory.text_scorer()

In [35]:
colCondenser_pipeline_ann_2020 = pt.transformer.SourceTransformer(colbert_2020) >>  colCondenser_factory.text_scorer()
colCondenser_pipeline_2020 = pt.transformer.SourceTransformer(dph_2020) >>  colCondenser_factory.text_scorer()

### RD3: Multi-Task Learning (MTL)   

In [36]:
mtl_pipeline_ann_2020 = pt.transformer.SourceTransformer(colbert_2020) >>  mtl_factory.text_scorer()
mtl_pipeline_2020 = pt.transformer.SourceTransformer(dph_2020) >>  mtl_factory.text_scorer()

In [ ]:
pt.Experiment(
    [mtl_pipeline_2020],
    pt.get_dataset('msmarco_passage').get_topics('test-2020'),
    pt.get_dataset('msmarco_passage').get_qrels('test-2020'),
    filter_by_qrels=True,
    names=['DPH >> ColBERT'],
    eval_metrics= [nDCG@10, AP(rel=2)@100, RR(rel=2)@10]
)

## **Experiment**

### DPH(Sparse Retrieval) Re-ranking

In [40]:
pt.Experiment(
    [colbert_pipeline_2020, cls_similarity_pipeline_2020, token_overlap_pipeline_2020, colCondenser_pipeline_2020, coltacl_pipeline_2020, mtl_pipeline_2020],
    pt.get_dataset('msmarco_passage').get_topics('test-2020'),
    pt.get_dataset('msmarco_passage').get_qrels('test-2020'),
    filter_by_qrels=True,
    names=['DPH >> ColBERT', 'DPH >> ColBERT-increase-cls-similarity', 'DPH >> ColBERT-add-token-overlap', 'DPH >> ColCondenser', 'DPH >> ColTaCL', 'DPH >> ColBERT-mtl-objective'],
    eval_metrics= [nDCG@10, AP(rel=2)@100, RR(rel=2)@10]
)

,name,nDCG@10,AP(rel=2)@100,RR(rel=2)@10
0,DPH >> ColBERT,0.657733,0.395494,0.835979
1,DPH >> ColBERT-increase-cls-similarity,0.668525,0.402908,0.853395
2,DPH >> ColBERT-add-token-overlap,0.655973,0.401742,0.833848
3,DPH >> ColCondenser,0.658674,0.393606,0.820216
4,DPH >> ColTaCL,0.663475,0.402527,0.822531
5,DPH >> ColBERT-mtl-objective,0.659804,0.382391,0.847737


### ColBERT(Dense Retrieval) Re-ranking

In [41]:
pt.Experiment(
    [colbert_pipeline_ann_2020, cls_similarity_pipeline_ann_2020, token_overlap_pipeline_ann_2020, colCondenser_pipeline_ann_2020, coltacl_pipeline_ann_2020, mtl_pipeline_ann_2020],
    pt.get_dataset('msmarco_passage').get_topics('test-2020'),
    pt.get_dataset('msmarco_passage').get_qrels('test-2020'),
    filter_by_qrels=True,
    names=['ANN-ColBERT >> ColBERT', 'ANN-ColBERT >> ColBERT-increase-cls-similarity', 'ANN-ColBERT >> ColBERT-add-token-overlap', 'ANN-ColBERT >> ColCondenser', 'ANN-ColBERT >> ColTaCL', 'ANN-ColBERT >> ColBERT-mtl-objective'],
    eval_metrics= [nDCG@10, AP(rel=2)@100, RR(rel=2)@10]
)

,name,nDCG@10,AP(rel=2)@100,RR(rel=2)@10
0,ANN-ColBERT >> ColBERT,0.693783,0.446686,0.842953
1,ANN-ColBERT >> ColBERT-increase-cls-similarity,0.702668,0.455801,0.856481
2,ANN-ColBERT >> ColBERT-add-token-overlap,0.671862,0.439425,0.803417
3,ANN-ColBERT >> ColCondenser,0.659519,0.434365,0.798148
4,ANN-ColBERT >> ColTaCL,0.669823,0.448219,0.836420
5,ANN-ColBERT >> ColBERT-mtl-objective,0.631162,0.396310,0.833848
